In [1]:
#导入库
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import random
from snownlp import SnowNLP


In [ ]:

#设置中文和本机请求头
options = webdriver.ChromeOptions()
#options.add_argument("lang=zh_CN.UTF-8")
#options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"')
browser = webdriver.Chrome()
#传入网页链接并打开
url = "https://guba.eastmoney.com/"
browser.get(url)
#等待时间
time.sleep(3)
#定位到搜索框
input_su = browser.find_element(By.CSS_SELECTOR,"#searchIn")
#传入点击指令
input_su.send_keys("601633")
#定位到搜索按钮并传入点击指令
su = browser.find_element(By.CSS_SELECTOR,"#search > div.search-bar > div > div.search-wrapper > div > div.submit._sptr")
su.click()
#定位窗口数量
windows = browser.window_handles
#点击后产生新的窗口，定位到新窗口
browser.switch_to.window(windows[-1])
#新用户
zhuansu = browser.find_element(By.CSS_SELECTOR,"#app > div.main.container > div.c_l > div.bar_item_list > div > a")
zhuansu.click()
#定位窗口数量
windows = browser.window_handles
#点击后产生新的窗口，定位到新窗口
browser.switch_to.window(windows[-1])
pinglun = {}
#设置while循环
i = 1
while i <25:
    #设置点击循环爬取，定位到所有li
    lilist = browser.find_elements(By.CSS_SELECTOR,".listitem")
    #遍历循环所有li，定位到标题和链接
    for li in lilist:
        a = li.find_element(By.CSS_SELECTOR,"td:nth-child(3) > div")
                                            #mainlist > div > ul > li.defaultlist > table > tbody > tr:nth-child(61) > td:nth-child(3) > div
        #获取标题和链接，并使用字典进行一一对应的保存
        title = a.text
        b = li.find_element(By.CSS_SELECTOR,"td:nth-child(3) > div >a")
        lianjie = b.get_attribute("href")
        pinglun[title] = lianjie
    #定位到下一页选项卡，并传入点击指令，循环加一
    nex = browser.find_element(By.CSS_SELECTOR,".nextp")
    nex.click()
    time.sleep(1)
    i = i+1  
#print(len(pinglun))
#保存字典为Excel
shuchu = pd.DataFrame(list(pinglun.items()))
shuchu.to_excel("guba.xlsx",header = ["标题","链接"],index = False)
#遍历储存链接的表格爬取内容
df = pd.read_excel(r"guba.xlsx")
neirong ={}
for i in range(len(df["标题"])):
    #将链接传入浏览器
    browser.get(df["链接"][i])
    time.sleep(random.randint(2,4))
    #进行try运行，防止报错中断程序
    try:
        e = browser.find_element(By.CSS_SELECTOR,"#newscontent > div.newstext")
        g = e.text
        neirong[df["标题"][i]] = g
        print(f"保存成功第{i}个")
    #当try内的代码报错的时候，运行except的代码
    except:
        print(f"第{i}个error")


In [ ]:
        
print(neirong)

#将字典储存为excel表格
baogao = pd.DataFrame(list(neirong.items()))
baogao.to_excel("neirong.xlsx",header = ["标题","内容"],index = False)
#合并两个表格
m1 = pd.read_excel("guba.xlsx",converters={"标题":str,"链接":str})
m2 = pd.read_excel("neirong.xlsx",converters={"标题":str,"内容":str})   
m = pd.merge(m1,m2,on =["标题"],how ='left') 
#输出合并后的内容
m.to_excel("pinglun.xlsx") 


In [ ]:

data =pd.read_excel("pinglun.xlsx",converters={"标题":str,"链接":str,"内容":str,"情绪":str})
data["情绪"].head(5)
#print(range(len(data["标题"])))
#遍历循环爬取到的股吧评论
for i in range(len(data["标题"])):
    #有些评论是非文字性的，无法获得评分，利用try函数进行替换
    try:
        data["情绪"][i] = SnowNLP(data["内容"][i]).sentiments
    except:
        #excel对0的数值进行替换
        data["情绪"][i] = "0"
data["情绪"].head(5)
#另存为
data.to_excel("QX.xlsx")

browser.quit()